In [1]:
!pip install beautifulsoup4 fitz

In [2]:
import re
from bs4 import BeautifulSoup
import requests

url = 'https://www.planalto.gov.br/ccivil_03/leis/l9099.htm'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')

# Remove content within <s>, <strike>, <del> tags
for tag in soup.find_all(['s', 'strike', 'del']):
    tag.decompose()  # Remove the tag and its content completely

if soup.body:
    cleaned_text = soup.body.get_text(separator=' ', strip=True)
else:
    cleaned_text = soup.get_text(separator=' ', strip=True)

#cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

with open('lei_9099_cleaned.txt', 'w', encoding='utf-8') as file:
    file.write(cleaned_text)

# The cleaned_text is now ready for chunking with LangChain
print(cleaned_text[:10000])

Presidência
    da República Casa Civil Subchefia para Assuntos Jurídicos LEI Nº 9.099, DE 26 DE SETEMBRO DE 1995. Mensagem de veto Vigência Dispõe sobre
    os Juizados Especiais Cíveis e Criminais e dá outras providências. O
PRESIDENTE DA REPÚBLICA Faço saber que o Congresso Nacional decreta e eu sanciono a
seguinte Lei: CAPÍTULO I Disposições Gerais Art. 1º Os Juizados Especiais Cíveis e
Criminais, órgãos da Justiça Ordinária, serão criados pela União, no Distrito
Federal e nos Territórios, e pelos Estados, para conciliação, processo, julgamento e
execução, nas causas de sua competência. Art. 2º O processo orientar-se-á pelos
critérios da oralidade, simplicidade, informalidade, economia processual e celeridade,
buscando, sempre que possível, a conciliação ou a transação. Capítulo II Dos Juizados Especiais Cíveis Seção I Da Competência Art. 3º O Juizado Especial Cível tem
competência para conciliação, processo e julgamento das causas cíveis de menor
complexidade, assim consideradas: 

In [4]:
url = 'http://normas.receita.fazenda.gov.br/sijut2consulta/link.action?idAto=1940'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')

# Remove content within <s>, <strike>, <del> tags
for tag in soup.find_all(['s', 'strike', 'del']):
    tag.decompose()  # Remove the tag and its content completely

if soup.body:
    cleaned_text = soup.body.get_text(separator=' ', strip=True)
else:
    cleaned_text = soup.get_text(separator=' ', strip=True)

cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

with open('lei_9099_cleaned.txt', 'w', encoding='utf-8') as file:
    file.write(cleaned_text)

# The cleaned_text is now ready for chunking with LangChain
print(cleaned_text[:10000])

NORMAS Contraste   Ato Declaratório Cosar nº 47, de 27 de novembro de 2000 (Publicado(a) no DOU de 06/12/2000, seção , página 5) Multivigente Vigente Original Relacional "Dispõe sobre código de Arrecadação de Receitas Federais." O COORDENADOR-GERAL DO SISTEMA DE ARRECADAÇÃO E COBRANÇA, no uso de suas atribuições, declara: Art. 1º O imposto de renda retido na fonte sobre prêmios obtidos em sorteio dos jogos de bingo permanente ou eventual, de que trata o inciso I, do artigo 14 do Decreto No 3.659, de 14 de novembro de 2000, deverá ser recolhido ao Tesouro Nacional mediante o Documento de Arrecadação de Receitas Federais - DARF, sob o código de receita 8673. Art. 2º O valor correspondente à parcela dos recursos arrecadados em sorteio dos jogos de bingo permanente ou eventual, destinado à União, para fomento do esporte e turismo, de que trata o inciso IV, do artigo 14 do Decreto No 3.659, de 14 de novembro de 2000, deverá ser recolhido ao Tesouro Nacional mediante o Documento de Arrecad

In [ ]:
import pandas as pd
import fitz  # PyMuPDF
import os
import re
from bs4 import BeautifulSoup
import requests

df = pd.read_excel("frequencia_normas_padronizadas - link.xlsx")

# Função para extrair texto de um PDF usando fitz
def extract_text_from_pdf(pdf_path):
    try:
        text = ""
        with fitz.open(pdf_path) as doc:
            for page in doc:
                text += page.get_text()
        return text.strip()
    except Exception as e:
        print(f"Erro ao extrair texto do PDF {pdf_path}: {e}")
        return ""

# Processar cada link do DataFrame
for index, row in df.iterrows():
    url = str(row["Link:"])
    norma_name = str(row['Norma']).replace('/', '')
    file_name_pdf = f"normas_novas_pdf/{norma_name}.pdf"
    file_name_txt = f"normas_novas_txt/{norma_name}.txt"

    # Garantir que os diretórios existam
    os.makedirs("normas_novas_pdf", exist_ok=True)
    os.makedirs("normas_novas_txt", exist_ok=True)

    if url.endswith(".pdf"):
        # Baixar o PDF
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_name_pdf, 'wb') as f:
                f.write(response.content)
            print(f"PDF baixado: {file_name_pdf}")

            # Extrair texto do PDF
            pdf_text = extract_text_from_pdf(file_name_pdf)
            if pdf_text:
                with open(file_name_txt, 'w', encoding='utf-8') as txt_file:
                    txt_file.write(pdf_text)
                print(f"Texto extraído e salvo em: {file_name_txt}")

            # Excluir o PDF
            os.remove(file_name_pdf)
            print(f"PDF excluído: {file_name_pdf}")
        else:
            print(f"Erro ao baixar o PDF: {url}")
    else:
        # Tratar URLs HTML
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            html_content = response.text

            # Processar HTML com BeautifulSoup
            soup = BeautifulSoup(html_content, 'html.parser')
            for tag in soup.find_all(['s', 'strike', 'del']):
                tag.decompose()

            if soup.body:
                cleaned_text = soup.body.get_text(separator=' ', strip=True)
            else:
                cleaned_text = soup.get_text(separator=' ', strip=True)

            # Limpar espaços extras
            cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

            # Salvar texto em arquivo
            with open(file_name_txt, 'w', encoding='utf-8') as txt_file:
                txt_file.write(cleaned_text)
            print(f"Texto HTML salvo em: {file_name_txt}")
        else:
            print(f"Erro ao acessar o link HTML: {url}")